In [ ]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import shutil
import os
import csv
import sys

import json
import datetime
import pandas as pd
import numpy as np
import talib as ta
import math

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(days=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_days, money, foot, back_days = 0, times = 0):
    global BACK
    all_data = []
    #time = time.replace(day=7, day=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(days=-14)
    back = datetime.timedelta(days=-back_days)
    
    force = datetime.timedelta(days=-BACK)
    
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + force

    date_from = date_to + datetime.timedelta(days=-period_days)
    for i in range(times+1):    
        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加
        
        date_to = date_from
        date_from = date_to + datetime.timedelta(days=-period_days)
#         print(date_from, date_to)

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.EMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
#     df["SMA5"]= ta.EMA(close, timeperiod=5)# / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
#     df["SMA10"]= ta.EMA(close, timeperiod=10) / close
    df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
#     df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
#     df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
#     df["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)/close
    
    
#     df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
#     df['Rate'] = df['Rate']*1000
#     df['true']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['class'] = ((df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')).apply(lambda x: classify_two(x))
    df['class'] = df['class'].shift(-1)
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
    df = df.drop('Open', axis=1)
    df = df.drop('High', axis=1)
    df = df.drop('Low', axis=1)
    df = df.drop('Close', axis=1)
    df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    return df



def get_csv(back_days, days, money):
    global MULTI
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_days=days, money=money, foot = FOOT , back_days=back_days, times=MULTI)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(days*MULTI)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_days, days, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(days)+'.csv'])
    
    return csvlist
        
def classify_two(x):
        if x<0:
            return -1
        elif 0<=x:
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        

def get_int_csv(moneylist):
    global TERM, DNUM
    shutil.rmtree('csv')
    os.mkdir('csv')
    array = get_moneys_csv(back_days=TERM, days=DNUM, moneylist = moneylist)
    return array

def make_bins(max_num, min_num, bin_num):
    global DIVIDE
    delta = max_num-min_num
    delta = delta/(DIVIDE-1)
    bins = []
    povit = min_num
    for i in range(DIVIDE-1):
        povit += delta
        bins.extend([povit])
#     print(bins)
    return bins

def digitize_data(df):
    bin_num = len(df)
    for i in range(len(df.columns)):
        col = df.columns.values.tolist()[i]
        if col == "class":
            continue
        elif col == "true":
            continue
        else:
            max_num = df[col].max()
            min_num = df[col].min()
#             max_num = df[col].quantile(0.75)
#             min_num = df[col].quantile(0.25)
            df[col] = np.digitize(df[col], make_bins(max_num, min_num, bin_num))
    return df

def grider(dataname):
    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    df = digitize_data(df)
#     print(df)
    return df
    
def make_table(line, money):
    global DIVIDE
    count = 0
    df = grider(money)
    print("Table data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)

#     table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    table = np.zeros((DIVIDE,DIVIDE, DIVIDE, DIVIDE))
    for y in range(len(df)):
        temp = []
        for x in range(len(df.columns)-1):
            temp.extend([df.iat[y,x]])
#         table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] += int(df.iat[y,len(df.columns)-1])
        table[temp[0]][temp[1]][temp[2]][temp[3]] += int(df.iat[y,len(df.columns)-1])
        count += 1
    print("Beyond LINE: ", end='')
    print(np.count_nonzero(table>line)/np.count_nonzero(table!=0))
    print("table count: "+str(count))
    return table

def buy_test(table, money):
    global FOOT, TERM, LINE, POSSESTION, PAY, PAYOUT, WIN, LOSE, COUNT
    df_now = get_now_data(datetime.datetime.now(), TERM, money, FOOT, back_days = 0, times = 0)
#     df_now['class'] = df_now['class'].shift(1)
    df_now = digitize_data(df_now)
    df_now = df_now.dropna()
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    count = [0,0,0]
    win = 0
    lose = 0
    glaph = []
    pos = POSSESTION
    history = []
    for y in range(len(df_now)):
        temp = []
        for x in range(len(df_now.columns)-1):
            temp.extend([df_now.iat[y,x]])
#         value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
        value = table[temp[0]][temp[1]][temp[2]][temp[3]]
        if value>=LINE: #HIGH予想
            count[0] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])>0): #実際HIGHなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
#                 table[temp[0]][temp[1]][temp[2]][temp[3]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]]
                continue
        elif value<=-LINE: #LOW予想
            count[1] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])<0): #実際LOWなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
#                 table[temp[0]][temp[1]][temp[2]][temp[3]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]]
                continue
        else:
            count[2] += 1
            history.extend([0])
        glaph.extend([pos])
    print("HIGH/LOW/STAY")
    print(count)
    print("win: "+str(win))
    print("lose: "+str(lose))
    
    WIN += win
    LOSE += lose
    COUNT += sum(count)
    
    if(win+lose==0):
        print("No Trade")
    else:
        print("Rate: "+str(win/(win+lose)))
    print("Posesstion: "+str(pos))
    POSSESTION += pos - POSSESTION
    
#     POSSESTION = 10000
    
#     if(POSSESTION < 0):
#         sys.exit()
#     plt.plot(list(range(0,len(glaph))), glaph)
#     plt.show()

    return win,lose, history

def main():
    global LINE, POSSESTION
    array = get_int_csv(MONEYS)
    w = 0
    l = 0
    for i in range(len(array)):
        print("")
        print("-----------------------")
        print(MONEYS[i])
        table = make_table(LINE, array[i])
#         print(table)
        win, lose, history = buy_test(table, MONEYS[i])
        w += win
        l += lose
#         with open('result.csv', 'a') as f:
#             writer = csv.writer(f)
#             writer.writerow([MONEYS[i], history])
    print("RESULT: "+str(POSSESTION))
    RESULT.extend([POSSESTION])
#     with open('result.csv', 'a') as f:
#         writer = csv.writer(f)
# #         writer.writerow([money, pos])
#         writer.writerow([POSSESTION, w/(w+l)])
    

FOOT = 'D'
DNUM = 4000
MULTI = 1  #datanum = DNUM * MULTI
DIVIDE = 5  #GRIDの数
LINE = 4  #トレードのライン(LINE以上，以下の時エントリー)
TERM = 100  #テスト期間
PAY = 1000  #支払額
PAYOUT = 1.9 #ペイアウト率
POSSESTION = 10000 #所持金
BACK = 0
RESULT = []
WIN = 0
LOSE = 0
COUNT = 0

MONEYS = [
        'AUD_JPY',
    
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
    
        'EUR_JPY',
        'EUR_USD',
    
#         'GBP_AUD',
    
        'GBP_JPY',
    
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
        'USD_JPY'
    ]

# MONEYS = [
#         'AUD_JPY',
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
#         'EUR_JPY',
#         'EUR_USD',
#         'GBP_AUD',
#         'GBP_JPY',
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
#         'USD_JPY'
#     ]

# np.set_printoptions(threshold=np.inf)

# for x in range(100):
#     print("")
#     print("TEST: "+ str(x))
#     main()
#     BACK += TERM
#     print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
#     print("Trade rate: "+str((WIN+LOSE)/COUNT))
#     print("Rate: "+str(WIN/(WIN+LOSE)))

DNUM = 100
DIVIDE = 2
BEST = 0
LINE = 2
for y in range(5):
    for i in range(10):
        for j in range(7):
            print("DNUM:"+str(DNUM)+" /DIVIDE:"+str(DIVIDE)+ " /LINE:" +str(LINE)+" /BEST:"+str(BEST))
            for x in range(100):
                print("")
                print("TEST: "+ str(x))
                main()
                BACK += TERM
                if(LOSE==0 and WIN==0):
                    LOSE = 1
                print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
                print("Trade rate: "+str((WIN+LOSE)/COUNT))
                print("Rate: "+str(WIN/(WIN+LOSE)))
                if(POSSESTION<0):
                    break
            with open('result.csv', 'a') as f:
                    writer = csv.writer(f)
                    if(BEST<WIN/(WIN+LOSE)):
                        BEST = WIN/(WIN+LOSE)
                        writer.writerow(["BEST", DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
                    else:
                        writer.writerow([DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
            WIN = 0
            LOSE = 0
            POSSESTION = 10000
            BACK = 0
            COUNT = 0
            DIVIDE += 1
        DNUM+=100
        DIVIDE = 2
        BACK = 0
    DNUM = 100
    LINE += 1
        

DNUM:100 /DIVIDE:2 /LINE:2 /BEST:0

TEST: 0
"AUD_JPY_2020-07-09_2020-10-17_100.csv", "EUR_JPY_2020-07-09_2020-10-17_100.csv", "EUR_USD_2020-07-09_2020-10-17_100.csv", "GBP_JPY_2020-07-09_2020-10-17_100.csv", "USD_JPY_2020-07-09_2020-10-17_100.csv", 
-----------------------
AUD_JPY
Table data
['2020-07-17 21:00'] to ['2020-07-03 21:00']
Beyond LINE: 0.3333333333333333
table count: 134

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[57, 0, 3]
win: 34
lose: 23
Rate: 0.5964912280701754
Posesstion: 17600.0

-----------------------
EUR_JPY
Table data
['2020-07-17 21:00'] to ['2020-07-03 21:00']
Beyond LINE: 0.25
table count: 134

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[57, 0, 3]
win: 30
lose: 27
Rate: 0.5263157894736842
Posesstion: 17600.0

-----------------------
EUR_USD
Table data
['2020-07-17 21:00'] to ['2020-07-03 21:00']
Beyond LINE: 0.25
table count: 134

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[56, 0, 4

"AUD_JPY_2020-07-09_2020-10-17_100.csv", "EUR_JPY_2020-07-09_2020-10-17_100.csv", "EUR_USD_2020-07-09_2020-10-17_100.csv", "GBP_JPY_2020-07-09_2020-10-17_100.csv", "USD_JPY_2020-07-09_2020-10-17_100.csv", 
-----------------------
AUD_JPY
Table data
['2019-12-28 22:00'] to ['2019-12-14 22:00']
Beyond LINE: 0.125
table count: 132

test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[9, 25, 29]
win: 17
lose: 17
Rate: 0.5
Posesstion: 21100.0

-----------------------
EUR_JPY
Table data
['2019-12-28 22:00'] to ['2019-12-14 22:00']
Beyond LINE: 0.2222222222222222
table count: 133

test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[6, 20, 37]
win: 18
lose: 8
Rate: 0.6923076923076923
Posesstion: 29300.0

-----------------------
EUR_USD
Table data
['2019-12-28 22:00'] to ['2019-12-14 22:00']
Beyond LINE: 0.18181818181818182
table count: 133

test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[29, 2, 32]
win: 19
lose: 12
Rate: 0.61290322580645

"AUD_JPY_2020-07-09_2020-10-17_100.csv", "EUR_JPY_2020-07-09_2020-10-17_100.csv", "EUR_USD_2020-07-09_2020-10-17_100.csv", "GBP_JPY_2020-07-09_2020-10-17_100.csv", "USD_JPY_2020-07-09_2020-10-17_100.csv", 
-----------------------
AUD_JPY
Table data
['2018-08-17 21:00'] to ['2018-08-03 21:00']
Beyond LINE: 0.1
table count: 134

test data
['2018-11-24 22:00'] to ['2019-02-19 22:00']
HIGH/LOW/STAY
[9, 19, 32]
win: 17
lose: 11
Rate: 0.6071428571428571
Posesstion: 9300.0

-----------------------
EUR_JPY
Table data
['2018-08-17 21:00'] to ['2018-08-03 21:00']
Beyond LINE: 0.0
table count: 134

test data
['2018-11-24 22:00'] to ['2019-02-19 22:00']
HIGH/LOW/STAY
[4, 20, 36]
win: 14
lose: 10
Rate: 0.5833333333333334
Posesstion: 11900.0

-----------------------
EUR_USD
Table data
['2018-08-17 21:00'] to ['2018-08-03 21:00']
Beyond LINE: 0.2222222222222222
table count: 134

test data
['2018-11-24 22:00'] to ['2019-02-19 22:00']
HIGH/LOW/STAY
[15, 6, 39]
win: 10
lose: 11
Rate: 0.47619047619047616


test data
['2020-07-17 21:00'] to ['2020-10-09 21:00']
HIGH/LOW/STAY
[7, 12, 42]
win: 12
lose: 7
Rate: 0.631578947368421
Posesstion: 16600.0
RESULT: 16600.0
Trade/Trade chance: 134/606
Trade rate: 0.22112211221122113
Rate: 0.5522388059701493

TEST: 2
"AUD_JPY_2020-07-09_2020-10-17_100.csv", "EUR_JPY_2020-07-09_2020-10-17_100.csv", "EUR_USD_2020-07-09_2020-10-17_100.csv", "GBP_JPY_2020-07-09_2020-10-17_100.csv", "USD_JPY_2020-07-09_2020-10-17_100.csv", 
-----------------------
AUD_JPY
Table data
['2019-12-28 22:00'] to ['2019-12-14 22:00']
Beyond LINE: 0.18181818181818182
table count: 132

test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[3, 2, 58]
win: 1
lose: 4
Rate: 0.2
Posesstion: 13500.0

-----------------------
EUR_JPY
Table data
['2019-12-28 22:00'] to ['2019-12-14 22:00']
Beyond LINE: 0.05
table count: 133

test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[14, 15, 34]
win: 13
lose: 16
Rate: 0.4482758620689655
Posesstion: 9200.0

---------


test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[8, 4, 51]
win: 3
lose: 9
Rate: 0.25
Posesstion: 8900.0
RESULT: 8900.0
Trade/Trade chance: 136/921
Trade rate: 0.1476655808903366
Rate: 0.5220588235294118

TEST: 3
"AUD_JPY_2020-07-09_2020-10-17_100.csv", "EUR_JPY_2020-07-09_2020-10-17_100.csv", "EUR_USD_2020-07-09_2020-10-17_100.csv", "GBP_JPY_2020-07-09_2020-10-17_100.csv", "USD_JPY_2020-07-09_2020-10-17_100.csv", 
-----------------------
AUD_JPY
Table data
['2019-09-21 21:00'] to ['2019-09-06 21:00']
Beyond LINE: 0.03333333333333333
table count: 134

test data
['2019-12-28 22:00'] to ['2020-03-25 21:00']
HIGH/LOW/STAY
[2, 4, 55]
win: 2
lose: 4
Rate: 0.3333333333333333
Posesstion: 6700.0

-----------------------
EUR_JPY
Table data
['2019-09-21 21:00'] to ['2019-09-06 21:00']
Beyond LINE: 0.0
table count: 135

test data
['2019-12-28 22:00'] to ['2020-03-25 21:00']
HIGH/LOW/STAY
[1, 7, 53]
win: 5
lose: 3
Rate: 0.625
Posesstion: 8200.0

-----------------------
EUR_USD

...

"AUD_JPY_2018-08-09_2020-10-17_800.csv", "EUR_JPY_2018-08-09_2020-10-17_800.csv", "EUR_USD_2018-08-09_2020-10-17_800.csv", "GBP_JPY_2018-08-09_2020-10-17_800.csv", "USD_JPY_2018-08-09_2020-10-17_800.csv", 
-----------------------
AUD_JPY
Table data
['2016-12-24 22:00'] to ['2016-12-09 22:00']
Beyond LINE: 0.1896551724137931
table count: 1126

test data
['2019-03-05 22:00'] to ['2019-05-30 21:00']
HIGH/LOW/STAY
[3, 1, 59]
win: 1
lose: 3
Rate: 0.25
Posesstion: 3900.0

-----------------------
EUR_JPY
Table data
['2016-12-24 22:00'] to ['2016-12-09 22:00']
Beyond LINE: 0.15492957746478872
table count: 1126

test data
['2019-03-05 22:00'] to ['2019-05-30 21:00']
HIGH/LOW/STAY
[1, 0, 62]
win: 0
lose: 1
Rate: 0.0
Posesstion: 2900.0

-----------------------
EUR_USD
Table data
['2016-12-24 22:00'] to ['2016-12-09 22:00']
Beyond LINE: 0.13157894736842105
table count: 1126

test data
['2019-03-05 22:00'] to ['2019-05-30 21:00']
HIGH/LOW/STAY
[0, 0, 63]
win: 0
lose: 0
No Trade
Posesstion: 2900.0



"AUD_JPY_2018-05-01_2020-10-17_900.csv", "EUR_JPY_2018-05-01_2020-10-17_900.csv", "EUR_USD_2018-05-01_2020-10-17_900.csv", "GBP_JPY_2018-05-01_2020-10-17_900.csv", "USD_JPY_2018-05-01_2020-10-17_900.csv", 
-----------------------
AUD_JPY
Table data
['2018-05-09 21:00'] to ['2018-04-25 21:00']
Beyond LINE: 0.42857142857142855
table count: 1269

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[34, 0, 26]
win: 21
lose: 13
Rate: 0.6176470588235294
Posesstion: 15900.0

-----------------------
EUR_JPY
Table data
['2018-05-09 21:00'] to ['2018-04-25 21:00']
Beyond LINE: 0.2727272727272727
table count: 1270

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[35, 20, 5]
win: 28
lose: 27
Rate: 0.509090909090909
Posesstion: 14100.0

-----------------------
EUR_USD
Table data
['2018-05-09 21:00'] to ['2018-04-25 21:00']
Beyond LINE: 0.09090909090909091
table count: 1269

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[13, 21, 26]
win: 1


test data
['2020-04-08 21:00'] to ['2020-07-03 21:00']
HIGH/LOW/STAY
[5, 0, 58]
win: 3
lose: 2
Rate: 0.6
Posesstion: 0.0
RESULT: 0.0
Trade/Trade chance: 257/921
Trade rate: 0.27904451682953313
Rate: 0.5058365758754864

TEST: 3
"AUD_JPY_2018-05-01_2020-10-17_900.csv", "EUR_JPY_2018-05-01_2020-10-17_900.csv", "EUR_USD_2018-05-01_2020-10-17_900.csv", "GBP_JPY_2018-05-01_2020-10-17_900.csv", "USD_JPY_2018-05-01_2020-10-17_900.csv", 
-----------------------
AUD_JPY
Table data
['2017-07-13 21:00'] to ['2017-06-29 21:00']
Beyond LINE: 0.4166666666666667
table count: 1273

test data
['2019-12-28 22:00'] to ['2020-03-25 21:00']
HIGH/LOW/STAY
[1, 1, 59]
win: 1
lose: 1
Rate: 0.5
Posesstion: -100.0

-----------------------
EUR_JPY
Table data
['2017-07-13 21:00'] to ['2017-06-29 21:00']
Beyond LINE: 0.17647058823529413
table count: 1274

test data
['2019-12-28 22:00'] to ['2020-03-25 21:00']
HIGH/LOW/STAY
[5, 8, 48]
win: 7
lose: 6
Rate: 0.5384615384615384
Posesstion: 200.0

-----------------------


test data
['2020-07-17 21:00'] to ['2020-10-09 21:00']
HIGH/LOW/STAY
[2, 0, 59]
win: 0
lose: 2
Rate: 0.0
Posesstion: -4200.0
RESULT: -4200.0
Trade/Trade chance: 132/606
Trade rate: 0.21782178217821782
Rate: 0.4696969696969697
DNUM:900 /DIVIDE:6 /LINE:2 /BEST:0.519753086419753

TEST: 0
"AUD_JPY_2018-05-01_2020-10-17_900.csv", "EUR_JPY_2018-05-01_2020-10-17_900.csv", "EUR_USD_2018-05-01_2020-10-17_900.csv", "GBP_JPY_2018-05-01_2020-10-17_900.csv", "USD_JPY_2018-05-01_2020-10-17_900.csv", 
-----------------------
AUD_JPY
Table data
['2018-05-09 21:00'] to ['2018-04-25 21:00']
Beyond LINE: 0.2
table count: 1269

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[10, 5, 45]
win: 11
lose: 4
Rate: 0.7333333333333333
Posesstion: 15900.0

-----------------------
EUR_JPY
Table data
['2018-05-09 21:00'] to ['2018-04-25 21:00']
Beyond LINE: 0.08695652173913043
table count: 1270

test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[4, 8, 48]
win: 5
lose: 7
Rate:


test data
['2020-10-24 21:00'] to ['2021-01-19 22:00']
HIGH/LOW/STAY
[2, 2, 57]
win: 0
lose: 4
Rate: 0.0
Posesstion: 5700.0
RESULT: 5700.0
Trade/Trade chance: 29/301
Trade rate: 0.09634551495016612
Rate: 0.4482758620689655

TEST: 1
"AUD_JPY_2018-05-01_2020-10-17_900.csv", "EUR_JPY_2018-05-01_2020-10-17_900.csv", "EUR_USD_2018-05-01_2020-10-17_900.csv", "GBP_JPY_2018-05-01_2020-10-17_900.csv", "USD_JPY_2018-05-01_2020-10-17_900.csv", 
-----------------------
AUD_JPY
Table data
['2018-01-27 22:00'] to ['2018-01-13 22:00']
Beyond LINE: 0.20833333333333334
table count: 1272

test data
['2020-07-17 21:00'] to ['2020-10-09 21:00']
HIGH/LOW/STAY
[4, 1, 56]
win: 3
lose: 2
Rate: 0.6
Posesstion: 6400.0

-----------------------
EUR_JPY
Table data
['2018-01-27 22:00'] to ['2018-01-13 22:00']
Beyond LINE: 0.16
table count: 1273

test data
['2020-07-17 21:00'] to ['2020-10-09 21:00']
HIGH/LOW/STAY
[4, 1, 56]
win: 3
lose: 2
Rate: 0.6
Posesstion: 7100.0

-----------------------
EUR_USD
Table data
['2